# Prefix Functions


## Substring search

As we know in computer world and bioinformatics in particular very often strings are used. You need to compare different DNA or RNA sequences with each other to find something. And the most famous task that you should was hear is find substring inside some string. For example classic word `S`='abracadabra' and we need find out is there substring `C`='bra' and if yes - where?

In this string you already see two variants of string 'bra' at second and ninth positions, but what to do with the text with the length about billions of characters (to compare billion of character is only about 1 Gb data, and Wikipedia contain about hundreds Gb of texts).

The most simple and direct way is compare the string we are looking for with each substring of the text. For our example above it will look like 'abr' != 'bra' -> 'bra' == 'bra' (good!) -> 'rac' != 'bra' ..... 'abr' != 'bra' -> 'bra' == 'bra' (good!). Sounds as a good solutions, but lets calculate the complexity of such algorithm.

In worst situation we should for each position inside text stops to check if our string is here. So, we have `N` symbols inside text and `M` symbols inside string, and if we will check `M` symbols for each position in text (`N` times) we spend `O(N*M)` steps to do this. Looks likes the linear complexity, but if our `M` will be near `N` (for example `N = 4*M`) we got the complexity equal to `O(N*N) = O(N^2)`. Not so good. And how always we need to do something with it.

## Suffix and Prefix

But before we create our algorithm let is discuss some interesting things calling suffix and prefix.

As you should know from natural language theory (I hope so) the `suffix` is the part of the world that is at the end of the word. For example for word 'abrakadabra' the suffix with the length 5 will be 'dabra', and with the length 3 'bra'.

![suffix](images/11-04/suffix.png)

So, for each string we have different suffix of different length that is actually our string but without some symbols at the beginning. Yea, a little boring, but wait some time.

`Prefix` - it is something like suffix but on the other side of the word. In other world it is the first `n` symbols of the string. For example for string '123456789' the prefix of length 4 will be '1234'. And the prefix of length 6 will be '123456'.

![prefix](images/11-04/prefix.png)

Ok, I hope it will be really simple, but how it can help us to find string inside text? For this let's calculate some special function.

## Prefix function

First of all, we need find the string inside some text. Usual prefix or suffix not help us to do this, but what if we write our string before text like `string#Ttttttteeeeeeeexxxxxxxstringttttttttt`, than our prefix with length of string will be exactly our string. And to find it inside the text we need find where the prefix occurs again in the text. Function, that calculate it for each position inside the text called prefix-function.

`The prefix function` is the function that for each position inside string calculate the maximum length of the suffix of substring \[0;i\] that matches suffix of this string. In other world we take our string length S, cut from it the peace (0;i) and calculate how many symbols at the beginning matches symbols at the end.

Let is look to our example 'abracadabra'. For each position we take the substring from the beginning to this position. So, for p(2) we calc 'ab', so a!=b and that is mean that prefix function is equal to zero for this case. for p(9) we process substring 'abracadab' and here we have that first two symbols 'ab' is equal to last two symbols of string 'ab', but if we try to take three symbols from each end we got 'abr'!='dab' that mean that the result of our suffix function is 2.

![prefix-function](images/11-04/prefix-function.png)

So, with prefix function we just try to match the beginning and the end of substring. And how you guessed already the simplest way to calculate it is direct calculation. So as in algorithm of search substring for each position we try to match maximum number of symbols at the position with the prefix of our string. But, as in previous algorithm, it can take from us about `O(n^2)` operations. However we did not learn dynamic programming for nothing. So let is use it here.

The only two parameters that we have is the string itself `S` and the position `i` where we want to calculate our suffix function `p(i)`. And let is just calculate this function for `p(i-1) = k` and try to use this answer for our task. Ok, let is assume that previous position of function was `k` mean that substrings (0;k-1) and (i-1-k;i-1) is equal. There can be some different situations and first of all:

![prefix_compare](images/11-04/prefix-comare.jpg)

- `p(i) <= p(i-1) + 1` - just because when we calculate this function at te next position we only can get that `s[p(i-1)] == s[i]` or `s[p(i-1)] != s[i]` mean that when we move to the next position we try to compare the two new symbols at them. So, if them is equal our prefix will become longer by one. And if not the prefix become shote which is written in the inequality.
- So, we already know what we should do if our symbols `s[p(i-1)] == s[i]` -> just write that out `p(i) = p(i-1) + 1`. But what we should do if they is not equal? Let is look to prefix function more closer. After we meet the symbols that are not equal we need to search new one prefix that match our current suffix. But wait, because all others symbols of suffix is equal to our prefix or in other words substrings `s[0;k]` and `s[i-k-1;i-1]` is equal we know that all symbols except the last one of `s[0;p(i)]` will be the same as in previous one function. So we can just calculate our function for position `p(i-1)` and find if it match with new symbols. In other words we take previously calculated result for our substring and check only last symbols. After that just recursively go to bottom till meat the situation when it match correctly of when we meet zero.

Now lets try it on example for better understanding. S='abceabcyabceabce' 'abce-abc-y-abce-abce'. I insert some '-' to make it easier to read. So, at some moment we come into position in forth part 'abceabce' we meet the symbol 'e' that is not equal to symbol y at the prefix. So, instead of recalculation of prefix function from zero we just check if the last prefix have the same 'little' prefix. In our situation 'abceabc' has the prefix equal to suffix 'abc', so instead of start from zero we try to check the substring from 'abc', because we already sure that it equal to our suffix. And if it matched we just write that `p(i) = p(p(i-1)) + 1` and continue calculation. In this situation we have that 'abc' + 'e' is equal to our prefix 'abce', so `p` from this position is equal to 4.

# Knuth–Morris–Pratt algorithm

So, lets back to our main task to find substring. One Idea we already discuss - we just write `string#fsfuyiuhaslfd` and try to calculate prefix function for all text. But take from us about `O(N)` additional memory what is to big. So, there is another idea of algorithm that can help us find substring and use only `O(M)` memeory, where M - the size of string that we try to find inside array.

![morris](images/11-04/morris.jpg)

The main idea of it is that we try to match our string with the text at position `i` like in first algorithm that we discuss. But instead of calculate it for each position we precalculate the prefix function for our string and use it to find the position inside string, where we can start to match our string from this position in text without going back to the beginning. The complexity of it still `O(N + M)`, but we not need search special symbol # to divide string from text and plus to this we spend only `O(M)` memory for algorithm.